In [ ]:
# -*- coding: utf-8 -*-
"""
IVDD binary classification (2-class: ivdd / normal) from DeepLabCut CSV (3-level header)
- 5 keypoints × (x,y) = 10 dims（likelihood 低品質点は補間）
- Windowing: SEQ_LEN=60, STRIDE=30
- Model: TimeDistributed(Dense->ReLU) -> LSTM -> LSTM -> Dense(2 logits)
- 学習率固定 + ReduceLROnPlateau（EarlyStoppingなし）
- 出力先はあなたの新ディレクトリ構成に準拠
    data/train/train_csv           ... 学習CSV
    data/train/fig                 ... 学習曲線（1枚）
    data/train/train1_model        ... 学習モデル(best/final)
    data/train/val_misclassified   ... 検証の誤分類ウィンドウ一覧CSV
"""

import os, re, glob, math, json
from datetime import datetime

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
from pathlib import Path


def detect_project_root() -> Path:
    # 1) 環境変数があれば最優先（推奨：固定化できます）
    env = os.environ.get("IVDD_PROJECT_ROOT")
    if env:
        p = Path(env).expanduser().resolve()
        if (p / "data").is_dir() or (p / "scripts").is_dir():
            return p

    # 2) スクリプト(.py)として動かす場合は __file__ から推定
    try:
        here = Path(__file__).resolve()
        if here.parent.name == "scripts":
            # …/scripts/ 配下のファイルならその親をrootに
            cand = here.parent.parent
            if (cand / "data").is_dir() or (cand / "scripts").is_dir():
                return cand
        # 直下に data や scripts があればそこをrootに
        if (here.parent / "data").is_dir() or (here.parent / "scripts").is_dir():
            return here.parent
    except NameError:
        # Notebook 実行時は __file__ が無い
        pass

    # 3) Notebook / どこで走らせてもOK：CWDから上に辿って scripts と data を持つ場所を探す
    cwd = Path.cwd()
    for cand in [cwd] + list(cwd.parents):
        if (cand / "scripts").is_dir() and (cand / "data").is_dir():
            return cand

    # 4) 最後の手段：scripts 直下ならその親、そうでなければCWD
    return cwd.parent if cwd.name == "scripts" else cwd

PROJ_ROOT = detect_project_root()
print(f"[INFO] Project root = {PROJ_ROOT}")

# 以降は PROJ_ROOT を使って固定パスを組み立てる
TRAIN_DIR     = PROJ_ROOT / "data" / "train"
TRAIN_CSV_DIR = TRAIN_DIR / "train_csv"
FIG_DIR       = TRAIN_DIR / "fig"
MODEL_DIR     = TRAIN_DIR / "train1_model"
VALERR_DIR    = TRAIN_DIR / "val_misclassified"


# os.path を使っている既存コードに合わせるなら str() を付けてください
CSV_GLOB = str(TRAIN_CSV_DIR / "*.csv")
for d in [FIG_DIR, MODEL_DIR, VALERR_DIR]:
    d.mkdir(parents=True, exist_ok=True)

# ========= 設定 =========
KEYPOINTS = [
    "tail_set",
    "right_tarsal",
    "right_paw",
    "left_tarsal",
    "left_paw",
]

USE_LIKELIHOOD      = False
MIN_KEEP_LIKELIHOOD = 0.6

SEQ_LEN   = 60
STRIDE    = 30
DIMS      = len(KEYPOINTS) * 2
BATCH_SIZE = 20
EPOCHS     = 50

LR = 3e-4
VAL_SPLIT_BY_FILE = True

CLASS_NAMES  = ["ivdd", "normal"]
CLASS_TO_IDX = {c:i for i,c in enumerate(CLASS_NAMES)}
N_CLASSES    = 2

tf.random.set_seed(42)
np.random.seed(42)

# ランごとユニークID（日時）
DATE_STR = datetime.now().strftime("%Y%m%d-%H%M%S")


# ========= ユーティリティ =========
def _norm_name(s: str) -> str:
    return "".join(ch for ch in s.lower() if ch not in " _-")

def _resolve_keypoints(all_bodyparts, requested):
    norm2orig = {}
    for bp in all_bodyparts:
        k = _norm_name(bp)
        if k not in norm2orig:
            norm2orig[k] = bp
    resolved, missing = [], []
    for req in requested:
        k = _norm_name(req)
        if k in norm2orig:
            resolved.append(norm2orig[k])
        else:
            missing.append(req)
    if missing:
        raise ValueError(f"指定キーポイントがCSVで見つかりません: {missing}\n利用可能: {all_bodyparts}")
    return resolved

# ===== ラベル推定（one/two/normal に対応） =====
IVDD_ALIASES   = {"ivdd", "one", "two"}
NORMAL_ALIASES = {"normal"}

def infer_label_from_filename(path: str) -> int:
    """
    ファイル名からラベルを推定。
    ルール（上から順に適用）:
      1) 拡張子を除いたファイル名を非英数字で分割し、トークンに
         - IVDD_ALIASES のいずれか（one/two/ivdd）が含まれ、
           かつ NORMAL_ALIASES が含まれない → ivdd
         - NORMAL_ALIASES（normal）が含まれ、
           かつ IVDD_ALIASES が含まれない → normal
      2) 1)で一意に決まらなければ、先頭トークンが IVDD_ALIASES / NORMAL_ALIASES のいずれかなら採用
      3) まだ未決なら親ディレクトリ名のトークンに同様のルールを適用（片方のみヒット時）
      4) それでも決まらなければ例外
    """
    name = os.path.basename(path).lower()
    stem = os.path.splitext(name)[0]

    # 非英数字で分割してトークン化（例: "one_10DLC_resnet50" -> ["one","10dlc","resnet50"]）
    tokens = [t for t in re.split(r'[^a-z0-9]+', stem) if t]
    token_set = set(tokens)

    has_ivdd   = any(t in IVDD_ALIASES   for t in token_set)
    has_normal = any(t in NORMAL_ALIASES for t in token_set)

    if has_ivdd and not has_normal:
        return CLASS_TO_IDX["ivdd"]
    if has_normal and not has_ivdd:
        return CLASS_TO_IDX["normal"]

    # フォールバック: 先頭トークン優先
    if tokens:
        if tokens[0] in IVDD_ALIASES:
            return CLASS_TO_IDX["ivdd"]
        if tokens[0] in NORMAL_ALIASES:
            return CLASS_TO_IDX["normal"]

    # さらに未決なら親ディレクトリで判定（片方のみヒット時）
    parent_tokens = [t for t in re.split(r'[^a-z0-9]+', os.path.dirname(path).lower()) if t]
    pset = set(parent_tokens)
    p_has_ivdd   = any(t in IVDD_ALIASES   for t in pset)
    p_has_normal = any(t in NORMAL_ALIASES for t in pset)
    if p_has_ivdd and not p_has_normal:
        return CLASS_TO_IDX["ivdd"]
    if p_has_normal and not p_has_ivdd:
        return CLASS_TO_IDX["normal"]

    # 判定不能
    raise ValueError(
        f"ラベル不明: {name}（推奨: 先頭を one_/two_/normal_ とする、またはどれかをトークンとして含める）"
    )

def read_dlc_5kp_xy(csv_path: str,
                    keypoints,
                    use_likelihood=False,
                    min_keep_likelihood=0.6):
    # 3段ヘッダ
    df = pd.read_csv(csv_path, header=[0,1,2], index_col=0)
    bodyparts = list({bp for (_, bp, _) in df.columns})
    use_kps = _resolve_keypoints(bodyparts, keypoints)

    # (x,y) 抽出
    cols = {}
    for bp in use_kps:
        cols[f"{bp}_x"] = df.xs((bp, "x"), level=[1,2], axis=1)
        cols[f"{bp}_y"] = df.xs((bp, "y"), level=[1,2], axis=1)
    X_df = pd.concat(cols.values(), axis=1)
    X_df.columns = list(cols.keys())

    if use_likelihood:
        for bp in use_kps:
            try:
                lcol = df.xs((bp, "likelihood"), level=[1,2], axis=1).values.flatten()
                low = lcol < min_keep_likelihood
                for c in [f"{bp}_x", f"{bp}_y"]:
                    vals = X_df[c].values
                    vals[low] = np.nan
                    X_df[c] = vals
            except KeyError:
                pass

    X_df = X_df.interpolate(method="linear", limit_direction="both", axis=0)
    X_df = X_df.bfill().ffill().fillna(0.0)
    X = X_df.values.astype(np.float32)  # (T,10)
    return X, use_kps

def zscore_per_file(X: np.ndarray, eps: float=1e-6) -> np.ndarray:
    mu = X.mean(axis=0, keepdims=True)
    sd = X.std(axis=0, keepdims=True)
    return (X - mu) / (sd + eps)

def make_windows(X: np.ndarray, seq_len: int, stride: int):
    n = X.shape[0]
    if n < seq_len:
        return np.empty((0, seq_len, X.shape[1]), dtype=X.dtype), []
    starts = list(range(0, n - seq_len + 1, stride))
    Xw = np.stack([X[s:s+seq_len] for s in starts], axis=0)
    return Xw, starts

def build_dataset(csv_paths, seq_len=SEQ_LEN, stride=STRIDE):
    X_list, y_list, file_ids, starts_list = [], [], [], []
    used_kps_any = None
    for p in csv_paths:
        y_lab = infer_label_from_filename(p)
        X_raw, used_kps = read_dlc_5kp_xy(
            p,
            keypoints=KEYPOINTS,
            use_likelihood=USE_LIKELIHOOD,
            min_keep_likelihood=MIN_KEEP_LIKELIHOOD
        )
        if X_raw.shape[1] != DIMS:
            raise ValueError(f"{os.path.basename(p)}: 取り出し次元 {X_raw.shape[1]} != 期待 {DIMS}")

        X_raw = zscore_per_file(X_raw)
        X_win, starts = make_windows(X_raw, seq_len, stride)  # (M,T,D), [M]
        if X_win.shape[0] == 0:
            print(f"[WARN] {os.path.basename(p)}: フレーム不足（{seq_len}未満）でスキップ")
            continue

        X_list.append(X_win)
        y_list.append(np.full((X_win.shape[0],), y_lab, dtype=np.int64))
        file_ids += [os.path.basename(p)] * X_win.shape[0]
        starts_list += starts
        used_kps_any = used_kps

    if not X_list:
        raise RuntimeError("データが作れませんでした。CSV と命名規則（ivdd/normal）を確認してください。")

    X = np.concatenate(X_list, axis=0)
    y = np.concatenate(y_list, axis=0)
    file_ids = np.array(file_ids)
    starts_arr = np.array(starts_list)
    print(f"[INFO] 使用キーポイント実名: {used_kps_any}")
    return X, y, file_ids, starts_arr

# ========= モデル =========
class LSTM_RNN(keras.Model):
    def __init__(self, n_input, n_hidden, n_classes):
        super().__init__()
        self.input_dense = keras.layers.Dense(n_hidden, activation='relu')
        self.time_dist   = keras.layers.TimeDistributed(self.input_dense)
        self.lstm1 = keras.layers.LSTM(n_hidden, return_sequences=True)
        self.lstm2 = keras.layers.LSTM(n_hidden)
        self.out   = keras.layers.Dense(n_classes)  # logits

    def call(self, x, training=False):
        x = self.time_dist(x)
        x = self.lstm1(x, training=training)
        x = self.lstm2(x, training=training)
        x = self.out(x)
        return x

class LSTMWithL2(LSTM_RNN):
    def __init__(self, n_input, n_hidden, n_classes, l2_lambda=1e-4):
        super().__init__(n_input, n_hidden, n_classes)
        self.l2_lambda   = l2_lambda
        self.loss_fn     = keras.losses.CategoricalCrossentropy(from_logits=True)
        self.metric_acc  = keras.metrics.CategoricalAccuracy(name="accuracy")
        self.metric_loss = keras.metrics.Mean(name="loss")

    @property
    def metrics(self):
        return [self.metric_loss, self.metric_acc]

    def compile(self, optimizer, **kwargs):
        super().compile(optimizer=optimizer, **kwargs)

    def _l2(self):
        if not self.trainable_variables:
            return 0.0
        return self.l2_lambda * tf.add_n([tf.nn.l2_loss(v) for v in self.trainable_variables])

    def train_step(self, data):
        if isinstance(data, (list, tuple)) and len(data) == 3:
            x, y, sample_weight = data
        else:
            x, y = data
            sample_weight = None
        with tf.GradientTape() as tape:
            logits = self(x, training=True)
            loss = self.loss_fn(y, logits, sample_weight=sample_weight) + self._l2()
        grads = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.trainable_variables))
        self.metric_loss.update_state(loss)
        self.metric_acc.update_state(y, logits, sample_weight=sample_weight)
        return {"loss": self.metric_loss.result(), "accuracy": self.metric_acc.result()}

    def test_step(self, data):
        if isinstance(data, (list, tuple)) and len(data) == 3:
            x, y, sample_weight = data
        else:
            x, y = data
            sample_weight = None
        logits = self(x, training=False)
        loss = self.loss_fn(y, logits, sample_weight=sample_weight) + self._l2()
        self.metric_loss.update_state(loss)
        self.metric_acc.update_state(y, logits, sample_weight=sample_weight)
        return {"loss": self.metric_loss.result(), "accuracy": self.metric_acc.result()}

# ========= データ読み込み =========
csv_files = sorted(glob.glob(CSV_GLOB))
if not csv_files:
    raise FileNotFoundError(f"学習CSVが見つかりません: {CSV_GLOB}")

X, y, file_ids, starts = build_dataset(csv_files, SEQ_LEN, STRIDE)
print("X:", X.shape, "y:", y.shape, "unique files:", len(np.unique(file_ids)))

# one-hot
y_oh = keras.utils.to_categorical(y, num_classes=N_CLASSES)

# ========= バリデーション分割（ファイル単位） =========
if VAL_SPLIT_BY_FILE:
    uniq_files = np.unique(file_ids)
    tr_files, va_files = train_test_split(uniq_files, test_size=0.3, random_state=42, shuffle=True)
    tr_mask = np.isin(file_ids, tr_files)
    va_mask = np.isin(file_ids, va_files)
    X_train, y_train = X[tr_mask], y_oh[tr_mask]
    X_val,   y_val   = X[va_mask], y_oh[va_mask]
    # バリデーションの元CSV名とウィンドウ開始フレームも保持
    val_file_names = file_ids[va_mask]
    val_starts     = starts[va_mask]
else:
    X_train, X_val, y_train, y_val, idx_train, idx_val = train_test_split(
        X, y_oh, np.arange(len(X)), test_size=0.2, random_state=42, stratify=y
    )
    val_file_names = file_ids[idx_val]
    val_starts     = starts[idx_val]

# ========= クラス不均衡対策 =========
cls_w = compute_class_weight("balanced", classes=np.arange(N_CLASSES), y=np.argmax(y_train, axis=1))
class_weight = {int(c): float(w) for c, w in enumerate(cls_w)}
print("class_weight:", class_weight)

# ========= モデル・学習 =========
n_hidden = 30
model = LSTMWithL2(n_input=DIMS, n_hidden=n_hidden, n_classes=N_CLASSES, l2_lambda=1e-4)
opt = keras.optimizers.Adam(learning_rate=LR)
model.compile(optimizer=opt)

best_path  = os.path.join(MODEL_DIR, f"ivdd_lstm_{DATE_STR}_best.keras")
final_path = os.path.join(MODEL_DIR, f"ivdd_lstm_{DATE_STR}_final.keras")

callbacks = [
    keras.callbacks.ModelCheckpoint(
        best_path, monitor="val_accuracy", mode="max",
        save_best_only=True, verbose=1
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", mode="min",
        factor=0.5, patience=5, min_lr=1e-6, verbose=1
    ),
]

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    class_weight=class_weight,
    verbose=1,
    callbacks=callbacks
)

# ========= 評価（バリデーション） & 誤分類CSV =========
y_val_prob = model.predict(X_val, batch_size=BATCH_SIZE)
y_val_pred = np.argmax(y_val_prob, axis=1)
y_val_true = np.argmax(y_val, axis=1)

print("\n[Validation] classification_report:")
print(classification_report(y_val_true, y_val_pred, target_names=CLASS_NAMES, digits=4))
cm = confusion_matrix(y_val_true, y_val_pred, labels=[0,1])
print("[Validation] confusion_matrix:\n", cm)

# 誤分類のみ抽出（ウィンドウ単位）
df_val = pd.DataFrame({
    "file": val_file_names,
    "start": val_starts,
    "true_idx": y_val_true,
    "pred_idx": y_val_pred,
    "true": [CLASS_NAMES[i] for i in y_val_true],
    "pred": [CLASS_NAMES[i] for i in y_val_pred],
    "p_ivdd": y_val_prob[:, CLASS_TO_IDX["ivdd"]],
    "p_normal": y_val_prob[:, CLASS_TO_IDX["normal"]],
})
df_mis = df_val[df_val["true_idx"] != df_val["pred_idx"]].copy()
mis_csv_path = os.path.join(VALERR_DIR, f"val_misclassified_{DATE_STR}.csv")
df_mis.to_csv(mis_csv_path, index=False, encoding="utf-8-sig")
print(f"[INFO] 誤分類ウィンドウ一覧を保存: {mis_csv_path} (rows={len(df_mis)})")

# ========= 学習曲線（loss & accuracy を1枚に） =========
fig_path = os.path.join(FIG_DIR, f"curve_{DATE_STR}.png")
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.plot(history.history["loss"], label="train")
plt.plot(history.history["val_loss"], label="val")
plt.title("Loss"); plt.legend()

plt.subplot(1,2,2)
plt.plot(history.history["accuracy"], label="train")
plt.plot(history.history["val_accuracy"], label="val")
plt.title("Accuracy"); plt.legend()

plt.tight_layout()
plt.savefig(fig_path, dpi=150)
plt.close()
print(f"[INFO] 学習曲線を保存: {fig_path}")

# ========= 最終モデル保存 =========
model.save(final_path)
print(f"[INFO] Final model saved to: {final_path}")
print(f"[INFO] Best model saved to:  {best_path}")

# ========= 学習ログ（テキスト）保存 =========
# 保存先ディレクトリ: data/train/logs/
LOG_DIR = os.path.join(TRAIN_DIR, "logs")
os.makedirs(LOG_DIR, exist_ok=True)

# タイムスタンプ（YYYYMMDD-HHMMSS）
date_str = datetime.now().strftime("%Y%m%d-%H%M%S")

# 使ったウィンドウ数・フレーム数（参考: フレーム数 = ウィンドウ数 × SEQ_LEN）
train_windows = int(X_train.shape[0])
val_windows   = int(X_val.shape[0])
train_frames  = int(train_windows * SEQ_LEN)
val_frames    = int(val_windows   * SEQ_LEN)

# 設定パラメータの記録（必要に応じて項目を増減してください）
params_text = []
params_text.append("[Training Parameters]")
params_text.append(f"RUN_ID              : {date_str}")
params_text.append(f"TRAIN_DIR           : {TRAIN_DIR}")
params_text.append(f"KEYPOINTS           : {KEYPOINTS}")
params_text.append(f"USE_LIKELIHOOD      : {USE_LIKELIHOOD}")
params_text.append(f"MIN_KEEP_LIKELIHOOD : {MIN_KEEP_LIKELIHOOD}")
params_text.append(f"SEQ_LEN             : {SEQ_LEN}")
params_text.append(f"STRIDE              : {STRIDE}")
params_text.append(f"DIMS                : {DIMS}")
params_text.append(f"N_HIDDEN            : {n_hidden if 'n_hidden' in globals() else 30}")
params_text.append(f"BATCH_SIZE          : {BATCH_SIZE}")
params_text.append(f"EPOCHS              : {EPOCHS}")
params_text.append(f"LR                  : {LR}")
params_text.append(f"VAL_SPLIT_BY_FILE   : {VAL_SPLIT_BY_FILE}")
params_text.append(f"CLASS_NAMES         : {CLASS_NAMES}")
params_text.append(f"class_weight        : {class_weight}")

# 各エポックのメトリクス（train/val の loss/accuracy）
hist = history.history
num_epochs = len(hist.get("loss", []))

epochs_text = []
epochs_text.append("\n[Per-epoch Metrics]")
epochs_text.append("epoch,loss,accuracy,val_loss,val_accuracy")
for i in range(num_epochs):
    tr_loss = hist.get("loss",         [None]*num_epochs)[i]
    tr_acc  = hist.get("accuracy",     [None]*num_epochs)[i]
    va_loss = hist.get("val_loss",     [None]*num_epochs)[i]
    va_acc  = hist.get("val_accuracy", [None]*num_epochs)[i]
    epochs_text.append(f"{i+1},{tr_loss:.6f},{tr_acc:.6f},{va_loss:.6f},{va_acc:.6f}")

# 最終 classification report（バリデーション）
report_text = []
report_text.append("\n[Validation Classification Report (window-level)]")
report_text.append(classification_report(y_val_true, y_val_pred, target_names=CLASS_NAMES, digits=4))

# サマリ（学習に使ったウィンドウ／フレーム数）
summary_text = []
summary_text.append("\n[Data Usage Summary]")
summary_text.append(f"train_windows: {train_windows}")
summary_text.append(f"val_windows  : {val_windows}")
summary_text.append(f"train_frames : {train_frames}")
summary_text.append(f"val_frames   : {val_frames}")

# まとめて .txt に保存
log_path = os.path.join(LOG_DIR, f"train_log_{date_str}.txt")
with open(log_path, "w", encoding="utf-8") as f:
    f.write("\n".join(params_text))
    f.write("\n")
    f.write("\n".join(summary_text))
    f.write("\n")
    f.write("\n".join(epochs_text))
    f.write("\n")
    f.write("\n".join(report_text))

print(f"[INFO] 学習ログを保存しました: {log_path}")



[INFO] Project root = c:\kanno\vscode\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master
[INFO] 使用キーポイント実名: ['tail_set', 'right_tarsal', 'right_paw', 'left_tarsal', 'left_paw']
X: (791, 60, 10) y: (791,) unique files: 115
class_weight: {0: 0.7, 1: 1.75}
Epoch 1/50
30/31 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5205 - loss: 0.7166
Epoch 1: val_accuracy improved from -inf to 0.47090, saving model to c:\kanno\vscode\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\data\train\train1_model\ivdd_lstm_20251222-132142_best.keras
31/31 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - accuracy: 0.5215 - loss: 0.7157 - val_accuracy: 0.4709 - val_loss: 0.7097 - learning_rate: 3.0000e-04
Epoch 2/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5100 - loss: 0.7057
Epoch 2: val_accuracy improved from 0.47090 to 0.52381, saving model to c:\kanno\vscode\RN